In [1]:
# [+] SparkSession 설정
from pyspark.sql import SparkSession

ss = SparkSession.builder.master('local').appName('mllib-first-exercise').getOrCreate()

## MLlib 예제: Logistic Regression
+ pyspark.ml.linalg: 선형대수(linear algebra) 관련 패키지
    + Vectors: 벡터 데이터 타입
+ pyspark.ml.classification: 분류 모델 관련 패키지
    + LogisticRegression: 가장 기본적인 분류 모델

In [3]:
# Vectors, LogisticRegression 임포트
from pyspark.ml.linalg import Vectors # ml이라는 패키지 안에 들어있는 것 -> 선형대수
from pyspark.ml.classification import LogisticRegression # ml이라는 패키지 안에 들어있는 것 -> 분류

In [4]:
"""
    데이터 정의: 레이블(정답)과 특징
    - dense(): 모든 값을 저장하는 벡터 생성
    - sparse(): 0이 아닌 값만 저장하는 벡터 생성
"""

train_data = [
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5])),
]
# 첫 번째 컬럼 : 레이블, 뒤의 연속된 값들은 특징들 / 1인지 0인지만 분류하는 것. 데이터가 무엇인지는 의미 X
# dense : 촘촘하다는 개념(반대로는 스파스? 희소. 예를 들어 column은 백만개인데 값은 10개)
# 희소 행렬은 그냥 index를 사용하는 것이 나음

In [12]:
# [+] 스키마 정의
schema = ['label', 'features']

In [14]:
# [+] DataFrame 생성
train_df = ss.createDataFrame(train_data, schema=schema) # 훈련데이터에 대한 데이터 프레임

In [15]:
train_df.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [21]:
"""
    [+] LogisticRegression (Estimator) 생성
    - maxIter: 최대 학습 횟수 매개변수
    - regParam: 정규화 매개변수
"""
lr = LogisticRegression(maxIter=10, regParam=0.001) # maxiter는 최대횟수 / regParam은 매개변수..?

In [22]:
# [+] 모델 학습 -> 보라색 선(두 클래스를 분할해주는 선)을 만들어주는 것
model = lr.fit(train_df)

In [23]:
model

LogisticRegressionModel: uid=LogisticRegression_c5187d0cee6d, numClasses=2, numFeatures=3

In [24]:
# 모델 테스트
test_data = [
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5])),
]

In [25]:
# [+] DataFrame 생성
test_df = ss.createDataFrame(test_data, schema=schema)

In [26]:
# [+] 모델 예측
predictions = model.transform(test_df)

In [28]:
# 예측결과 출력
predictions.show()
# rawPrediction은 중간 결과 보여주는 것

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0|[-1.0,1.5,1.3]|[-10.024339064172...|[4.43063136183005...|       1.0|
|  0.0|[3.0,2.0,-0.1]|[8.88090306449148...|[0.99986100075734...|       0.0|
|  1.0|[0.0,2.2,-1.5]|[-6.7315196407454...|[0.00119129819168...|       1.0|
+-----+--------------+--------------------+--------------------+----------+



In [ ]:
# Spark 이용해서 머신러닝 과정 한 것